<a href="https://colab.research.google.com/github/camrobertson/FDA_SupplementModel/blob/master/FDA_sup_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ML analysis to predict timing/nature of supplements following BLA/NDA submission

Input from drugs@fda.com
Highlight variables BLA/NDA, Date of Submission,, Therapeutic Area (review division), priority review, breakthrough therapy, accelerated approval
See what happens (start w/ RFS)

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib as mpl #only needed to determine Matplotlib version number
import csv

In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#works to import data
Products_df = pd.read_csv('/content/gdrive/My Drive/FDA_data/target/Products.txt', '\t', low_memory = False, error_bad_lines = False, encoding = "utf8")
Products_df.describe()
Products_df

In [0]:
#works to import data
Submissions_df = pd.read_csv('/content/gdrive/My Drive/FDA_data/target/Submissions.txt', '\t', low_memory = False, error_bad_lines = False, encoding='ISO-8859-1')
Submissions_df.describe()

In [0]:
#works to import data
Applications_df = pd.read_csv('/content/gdrive/My Drive/FDA_data/target/Applications.txt', '\t', low_memory = False, error_bad_lines = False, encoding='ISO-8859-1')
Applications_df.describe()

In [0]:
Master_df = Submissions_df.set_index('ApplNo').join(Applications_df.set_index('ApplNo'))

In [0]:
Master_df['SubmissionType'] = Master_df['SubmissionType'].map({'ORIG': 0, 'SUPPL': 1})

In [0]:
Master_df['ApplType'] = Master_df['ApplType'].map({'NDA': 0, 'BLA': 1})

In [0]:
Master_df.fillna(0, inplace = True)

In [0]:
Master_df['SubmissionClassCodeID', 'ApplType', 'ReviewPriority'] = Master_df['SubmissionClassCodeID', 'ApplType', 'ReviewPriority'].astype(int)

In [0]:
Master_df['ApplType'] = Master_df['ApplType'].astype(int)

In [0]:
Master_df['SubmissionClassCodeID'] = Master_df['SubmissionClassCodeID'].astype(int)

In [0]:
Master_df['ReviewPriority'] = Master_df['ReviewPriority'].astype(int)

In [0]:
Master_df

In [0]:
Master_df['SubmissionType'] = Master_df['SubmissionType'].map({'ORIG': 0, 'SUPPL': 1})

In [114]:
Submissions_df


,ApplNo,SubmissionClassCodeID,SubmissionType,SubmissionNo,SubmissionStatus,SubmissionStatusDate,SubmissionsPublicNotes,ReviewPriority
0,4,19.0,ORIG,1,AP,1969-07-16 00:00:00,NaN,UNKNOWN
1,4,3.0,SUPPL,10,AP,1980-05-08 00:00:00,NaN,NaN
2,4,3.0,SUPPL,11,AP,1987-05-26 00:00:00,NaN,NaN
3,159,NaN,ORIG,1,AP,1939-03-09 00:00:00,NaN,NaN
4,159,3.0,SUPPL,3,AP,1986-12-09 00:00:00,NaN,NaN
5,159,3.0,SUPPL,4,AP,1986-05-28 00:00:00,NaN,NaN
6,415,NaN,ORIG,1,AP,1939-02-27 00:00:00,NaN,NaN
7,552,19.0,ORIG,1,AP,1939-02-09 00:00:00,NaN,UNKNOWN
8,552,21.0,SUPPL,7,AP,1975-07-28 00:00:00,NaN,NaN
9,552,21.0,SUPPL,8,AP,1975-07-28 00:00:00,NaN,NaN


In [102]:
list(Master_df.columns)



['SubmissionClassCodeID',
 'SubmissionType',
 'SubmissionNo',
 'SubmissionStatus',
 'SubmissionStatusDate',
 'SubmissionsPublicNotes',
 'ReviewPriority',
 'ApplType',
 'ApplPublicNotes',
 'SponsorName']